In [ ]:
import sys
import psycopg2
import pandas as pd

## Requirements
```
pip install psycopg2-binary
pip install pandas
```

## Objective

Provide a simple database for logging machines data. This allows for the logging of machine data with a connector. This database is in a format that can be easily consumed by Sight Machine's AI data pipeline.

## Limitations and Conciderations

The schema provided is an event log of sensors being recorded. Therea are no assumptions around modeling of sensor names or validation of new sensors - that is left of the application code (and with the right amount of effort could be incorporated into this schema). It is also important to concider that we are using Postgres as our logging mechenism (and with this notebook Postgres on Docker). There are strategies to [increase write performance of Postgres](https://markandruth.co.uk/2016/01/08/how-we-tweaked-postgres-upsert-performance-to-be-2-3-faster-than-mongodb).

## Warrenty and Support

PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND

In [ ]:
print('Python Version:')
print(sys.version)
print('-'*80)
print('psycopg2 version:')
print(psycopg2.__version__)
print('-'*80)
print('pandas version:')
print(pd.__version__)
print('-'*80)

## CHANGE ME
This is the host IP that the docker container is running on:

In [ ]:
hostname = "10.211.55.27"

In [ ]:
conn = psycopg2.connect(host=hostname,database="simple_tag_logging", user="postgres", password="postgres")

In [ ]:
cur = conn.cursor()

In [ ]:
DROP = True

In [ ]:
if DROP:
    cur.execute("""DROP TABLE IF EXISTS tag_values;""")
    DROP = False
cur.execute("""
    CREATE TABLE IF NOT EXISTS tag_values
        (
            id SERIAL PRIMARY KEY,
            read_time TIMESTAMP NOT NULL,
            sensor_name TEXT NOT NULL,
            sensor_value VARCHAR(300) NOT NULL,
            sensor_location VARCHAR(80),
            sensor_datatype VARCHAR(80)
        );
""")

In [ ]:
cur.execute("CREATE INDEX tag_read_time ON tag_values(read_time DESC);")

In [ ]:
conn.commit()
cur.close()

In [ ]:
pd.read_sql("SELECT * FROM information_schema.tables WHERE table_schema='public'", conn)

In [ ]:
pd.read_sql("""SELECT *, pg_size_pretty(pg_relation_size(indexrelname::text))
    FROM pg_stat_all_indexes 
    WHERE schemaname = 'public'""", conn)